## Импорты

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchaudio

import yt_dlp
import subprocess

import os
import pickle
import gc
import json
import locale
import re
import tqdm.notebook as tqdm
from urllib.parse import urlparse
import requests
import math

import av
from huggingface_hub import hf_hub_download

from transformers import VivitImageProcessor, VivitModel
from transformers import AutoImageProcessor, VideoMAEModel
from transformers import TimesformerConfig, TimesformerModel
from transformers import XCLIPProcessor, XCLIPModel
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from transformers import BitsAndBytesConfig, LlavaNextVideoForConditionalGeneration, LlavaNextVideoProcessor
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

import librosa
from moviepy.editor import VideoFileClip
import ast
import openunmix

from minio import Minio
from minio.error import S3Error

import hydra
import soundfile as sf
from omegaconf import OmegaConf

import logging
from typing import List

import numpy as np
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

locale.getpreferredencoding = lambda: "UTF-8"

logging.basicConfig(
    filename='vivit_inference.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

Константы

In [ ]:
CATEGORIES_ENG = """Auto-moto,
Anime,
Audiobooks,
Business,
Video games,
Interview,
Art,
Movie,
Beauty,
Cooking,
Life Hacks,
Music,
Cartoons,
News,
Training,
Hunting and fishing,
Politics,
Psychology,
Journeys,
Serials,
Sport,
Humor,
Lifestyle,
Realty,
Health,
Nature,
Design,
Machinery and equipment,
Business and entrepreneurship,
Culture,
Religion,
Construction and renovation,
Garden and vegetable garden,
Food,
Entertainment,
Esotericism,
The science,
Audio,
Technology and the Internet,
TV shows,
For children,
Hobby,
Various,
Animals,
News and media,
Films,
Bloggers,
"""

CATEGORIES_RUS = """
Авто-мото
Аниме
Аудиокниги
Бизнес
Видеоигры
Интервью
Искусство
Кино
Красота
Кулинария
Лайфхаки
Музыка
Мультфильмы
Новости
Обучение
Охота_и_рыбалка
Политика
Психология
Путешествия
Сериалы
Спорт
Юмор
Лайфстайл
Недвижимость
Здоровье
Природа
Дизайн
Техника_и_оборудование
Бизнес_и_предпринимательство
Культура
Религия
Строительство_и_ремонт
Сад_и_огород
Еда
Развлечения
Эзотерика
Наука
Аудио
Технологии_и_интернет
Телепередачи
Детям
Хобби
Разное
Животные
Новости_и_СМИ
Фильмы
Блогеры
"""


RUS_TEXT_PROMPT = [
    f"Видео принадлежит категории '{x}'" for x in CATEGORIES_RUS.split() if x.strip() != ''
]
ENG_TEXT_PROMPT = [
    f"Video belongs to category '{x}'" for x in CATEGORIES_ENG.split(',\n') if x.strip() != ''
]

### Audio

#### Auxiliary functions

In [ ]:
def extract_audio_from_video(video_path, output_audio_path="temp_audio.wav", duration=60):
    video_clip = VideoFileClip(video_path)

    if video_clip is None:
        return None
    elif video_clip.audio is None:
        video_clip.close()
        return None

    audio_clip = video_clip.audio.subclip(0, min(duration, video_clip.duration))
    audio_clip.write_audiofile(output_audio_path, codec='pcm_s16le', ffmpeg_params=['-ac', '1'])

    # Close the clip to release resources
    video_clip.close()
    audio_clip.close()

    return output_audio_path


def speech_file_to_array_fn(batch, duration: int, sampling_rate: int):
    # Extract the audio from the mp4 file
    audio_path = extract_audio_from_video(batch["path"], duration=duration)

    if audio_path is None:
        return None

    # Load the extracted audio using librosa
    speech_array, sampling_rate = librosa.load(audio_path, sr=sampling_rate)

    # Prepare for Wav2Vec2 inference
    batch["speech"] = speech_array
    return batch


def extract_category_id(ctgry: str) -> int:
    return int(float(ctgry[:3]))


class SpecScaler(torch.nn.Module):
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return torch.log(x.clamp_(1e-9, 1e9))


def instantiate_giga_am(dirname: str, device: str):
    encoder_config = dirname+"encoder_config.yaml"
    model_weights = dirname+"ssl_model_weights.ckpt"
    conf = OmegaConf.load(encoder_config)

    encoder = hydra.utils.instantiate(conf.encoder)
    ckpt = torch.load(model_weights, map_location="cpu")
    encoder.load_state_dict(ckpt, strict=True)
    encoder.to(device)
    encoder.eval()

    feature_extractor = hydra.utils.instantiate(conf.feature_extractor)

    return encoder, feature_extractor


def process_gigaam(
    video_path: str,
    encoder,
    feature_extractor,
    device,
    audio_duration: int = 60,
    audio_sampling_rate: int = 16000
) -> torch.Tensor:
    try:
        test_sample = {"path": video_path}
        test_sample = speech_file_to_array_fn(test_sample, audio_duration, audio_sampling_rate)
        features = feature_extractor(torch.tensor(test_sample['speech']).float()).to(device)

        with torch.no_grad():
            encoded, _ = encoder.forward(
                audio_signal=features.unsqueeze(0),
                length=torch.tensor([features.shape[-1]]).to(device),
            )

        output = encoded.mean(dim=2).squeeze().cpu()
        return output

    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        print("An unexpected error occurred:")
        print(e)

#### Inference Giga-AM

In [ ]:
DATA_DIR = './data'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
encoder, feature_extractor = instantiate_giga_am('models/giga_am/', device)
embeddings_gigaam = []

for data_subdir in tqdm.tqdm(os.listdir(DATA_DIR)):
    fullpath = os.path.join(DATA_DIR, data_subdir)

    if not os.path.isdir(fullpath):
        continue

    video_files = [f for f in os.listdir(fullpath) if f.lower().endswith('.mp4')]
    logging.info(f"Found {len(video_files)} video(s) in directory '{data_subdir}'.")

    for video_file in video_files:
        file_path = os.path.join(fullpath, video_file)
        output = process_gigaam(file_path, encoder, feature_extractor, device)

        if output is not None:
            logging.info(f"Successfully processed video: {video_file}")
            embeddings_gigaam.append((video_file, extract_category_id(data_subdir), output))
        else:
            logging.warning(f"Failed to process video: {video_file}")

gigaam_df = EmbeddingStorage(
    filenames=list(map(lambda x: x[0], embeddings_gigaam)),
    labels=list(map(lambda x: x[1], embeddings_gigaam)),
    embeddings=list(map(lambda x: x[2], embeddings_gigaam)),
)
gigaam_df.save_to_file('data/embeddings/videomae.npz')

# gigaam_df = pd.DataFrame(data=embeddings_gigaam, columns=['videoname', 'label', 'emb'])
# gigaam_df.to_csv('data/embeddings/gigaam.csv', index=False)

[NeMo W 2024-09-25 19:56:11 nemo_logging:349] /tmp/ipykernel_7454/3025371214.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
      ckpt = torch.load(model_weights, map_lo

  0%|          | 0/48 [00:00<?, ?it/s]

MoviePy - Writing audio in temp_audio.wav





chunk:   0%|                                                                                                         | 0/1324 [00:00<?, ?it/s, now=None]


chunk:  65%|████████████████████████████████████████████████████████████▉                                | 867/1324 [00:00<00:00, 8663.56it/s, now=None]


                                                                                                                                                        

MoviePy - Done.
MoviePy - Writing audio in temp_audio.wav


...

MoviePy - Done.
An unexpected error occurred:
'NoneType' object has no attribute 'subclip'
MoviePy - Writing audio in temp_audio.wav





chunk:   0%|                                                                                                         | 0/1324 [00:00<?, ?it/s, now=None]


chunk:  65%|████████████████████████████████████████████████████████████▎                                | 858/1324 [00:00<00:00, 8578.31it/s, now=None]


                                                                                                                                                        

MoviePy - Done.
MoviePy - Writing audio in temp_audio.wav





chunk:   0%|                                                                                                          | 0/859 [00:00<?, ?it/s, now=None]


chunk:  96%|██████████████████████████████████████████████████████████████████████████████████████████▍   | 826/859 [00:00<00:00, 8258.37it/s, now=None]


                                                                                                                                                        

MoviePy - Done.


#### Raw notes on GigaAM-RNNT

In [ ]:
from nemo.collections.asr.models import EncDecRNNTBPEModel
from nemo.collections.asr.modules.audio_preprocessing import (
    AudioToMelSpectrogramPreprocessor as NeMoAudioToMelSpectrogramPreprocessor,
)
from nemo.collections.asr.parts.preprocessing.features import (
    FilterbankFeaturesTA as NeMoFilterbankFeaturesTA,
)

[NeMo W 2024-09-26 14:23:18 nemo_logging:349] /home/eduard/Рабочий стол/video-tagging/video-tag/lib/python3.10/site-packages/megatron/core/tensor_parallel/layers.py:254: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-09-26 14:23:18 nemo_logging:349] /home/eduard/Рабочий стол/video-tagging/video-tag/lib/python3.10/site-packages/megatron/core/tensor_parallel/layers.py:265: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-09-26 14:23:18 nemo_logging:349] /home/eduard/Рабочий стол/video-tagging/video-tag/lib/python3.10/site-packages/megatron/core/tensor_parallel/layers.py:325: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      

Download config, weights, tokenizer

In [ ]:
# !wget https://n-ws-q0bez.s3pd12.sbercloud.ru/b-ws-q0bez-jpv/GigaAM/rnnt_model_weights.ckpt
# !wget https://n-ws-q0bez.s3pd12.sbercloud.ru/b-ws-q0bez-jpv/GigaAM/rnnt_model_config.yaml
# !wget https://n-ws-q0bez.s3pd12.sbercloud.ru/b-ws-q0bez-jpv/GigaAM/example.wav
# !wget https://n-ws-q0bez.s3pd12.sbercloud.ru/b-ws-q0bez-jpv/GigaAM/long_example.wav
# !wget https://n-ws-q0bez.s3pd12.sbercloud.ru/b-ws-q0bez-jpv/GigaAM/tokenizer_all_sets.tar
# !tar -xf tokenizer_all_sets.tar && rm tokenizer_all_sets.tar

Modules for feature Extraction

In [ ]:
class FilterbankFeaturesTA(NeMoFilterbankFeaturesTA):
    def __init__(self, mel_scale: str = "htk", wkwargs=None, **kwargs):
        if "window_size" in kwargs:
            del kwargs["window_size"]
        if "window_stride" in kwargs:
            del kwargs["window_stride"]

        super().__init__(**kwargs)

        self._mel_spec_extractor: torchaudio.transforms.MelSpectrogram = (
            torchaudio.transforms.MelSpectrogram(
                sample_rate=self._sample_rate,
                win_length=self.win_length,
                hop_length=self.hop_length,
                n_mels=kwargs["nfilt"],
                window_fn=self.torch_windows[kwargs["window"]],
                mel_scale=mel_scale,
                norm=kwargs["mel_norm"],
                n_fft=kwargs["n_fft"],
                f_max=kwargs.get("highfreq", None),
                f_min=kwargs.get("lowfreq", 0),
                wkwargs=wkwargs,
            )
        )


class AudioToMelSpectrogramPreprocessor(NeMoAudioToMelSpectrogramPreprocessor):
    def __init__(self, mel_scale: str = "htk", **kwargs):
        super().__init__(**kwargs)
        kwargs["nfilt"] = kwargs["features"]
        del kwargs["features"]
        self.featurizer = (
            FilterbankFeaturesTA(  # Deprecated arguments; kept for config compatibility
                mel_scale=mel_scale,
                **kwargs,
            )
        )

Inference example

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = EncDecRNNTBPEModel.from_config_file("./models/giga_am/rnnt_model_config.yaml")
ckpt = torch.load("./models/giga_am/rnnt_model_weights.ckpt", map_location="cpu")
model.load_state_dict(ckpt, strict=False)
model.eval()
model = model.to(device)

[NeMo I 2024-09-26 14:40:59 mixins:172] Tokenizer SentencePieceTokenizer initialized with 512 tokens


[NeMo W 2024-09-26 14:40:59 audio_to_text_dataset:830] Could not load dataset as `manifest_filepath` was None. Provided config : {'shuffle': False, 'manifest_filepath': None}


[NeMo I 2024-09-26 14:40:59 features:289] PADDING: 0
[NeMo I 2024-09-26 14:41:00 rnnt_models:217] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}


[NeMo W 2024-09-26 14:41:00 nemo_logging:349] /tmp/ipykernel_206253/913905516.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
      ckpt = torch.load("./models/giga_am/rnn

In [ ]:
with torch.no_grad():
    outp = model.transcribe(["./example.wav"])[0]
    print(outp)

with torch.no_grad():
    outp = model.transcribe(["./long_example.wav"])[0]
    print(outp)

with torch.no_grad():
    outp = model.transcribe(["./temp_audio.wav"])[0]
    print(outp)

file_path_1 = 'example.wav'
audio, sr = librosa.load(file_path_1, sr=None, mono=False)
print(audio.shape)

file_path_2 = 'long_example.wav'
audio, sr = librosa.load(file_path_2, sr=None, mono=False)
print(audio.shape)

file_path_3 = 'temp_audio.wav'
audio, sr = librosa.load(file_path_3, sr=None, mono=False)
print(audio.shape)

#### Inference Whisper-V3

In [ ]:
def instantiate_whisper(device: str = 'cuda', torch_dtype = torch.float16):
    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        "models/whisper_model", torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
    ).to(device)
    processor = AutoProcessor.from_pretrained("models/whisper_processor")

    model.eval()

    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch_dtype,
        device=device,
        chunk_length_s=30,
        stride_length_s=5
    )

    return model, processor, pipe


def process_whisper(
    video_path: str,
    model,
    processor,
    asr_pipe,
    device = 'cuda',
    torch_dtype = torch.float16,
    audio_duration: int = 60,
    audio_sampling_rate: int = 16000
) -> torch.Tensor:
    try:
        inp_sample = {"path": video_path}
        test_sample = speech_file_to_array_fn(inp_sample, audio_duration, audio_sampling_rate)

        if test_sample is None:
            return None, None

        inputs = processor(test_sample['speech'], sampling_rate=audio_sampling_rate, return_tensors="pt")
        input_features = inputs.input_features.type(torch_dtype).to(device)

        with torch.no_grad():
            encoder_outputs = model.model.encoder(input_features)

        output = encoder_outputs.last_hidden_state.mean(dim=1).squeeze().cpu()

        sample = {'raw': test_sample['speech'], 'sampling_rate': audio_sampling_rate}
        decoded_speech = asr_pipe(sample)

        return output, decoded_speech['text']

    except Exception as e:
        logging.error(f"Unexpected error: {e}")
        print("An unexpected error occurred:")
        print(e)

In [ ]:
DATA_DIR = './data'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model, processor, asr_pipe = instantiate_whisper(device, torch_dtype)
embeddings_whisper = []

for data_subdir in tqdm.tqdm(os.listdir(DATA_DIR)):
    fullpath = os.path.join(DATA_DIR, data_subdir)

    if not os.path.isdir(fullpath):
        continue

    video_files = [f for f in os.listdir(fullpath) if f.lower().endswith('.mp4')]
    logging.info(f"Found {len(video_files)} video(s) in directory '{data_subdir}'.")

    for video_file in video_files:
        file_path = os.path.join(fullpath, video_file)
        output, recov_text = process_whisper(file_path, model, processor, asr_pipe, device, torch_dtype)

        if output is not None:
            logging.info(f"Successfully processed video: {video_file}")
            embeddings_whisper.append((video_file, extract_category_id(data_subdir), output.numpy(), recov_text))
        else:
            logging.warning(f"Failed to process video: {video_file}")

whisper_df = EmbeddingStorage(
    filenames=list(map(lambda x: x[0], embeddings_whisper)),
    labels=list(map(lambda x: x[1], embeddings_whisper)),
    embeddings=list(map(lambda x: x[2], embeddings_whisper)),
)
whisper_df.save_to_file('data/embeddings/whisper.npz')

whisper_text_df = EmbeddingStorage(
    filenames=list(map(lambda x: x[0], embeddings_whisper)),
    labels=list(map(lambda x: x[1], embeddings_whisper)),
    embeddings=list(map(lambda x: x[3], embeddings_whisper)),
)
whisper_text_df.save_to_file('data/embeddings/whisper_text.npz')

# whisper_df = pd.DataFrame(data=embeddings_whisper, columns=['videoname', 'label', 'emb', 'text'])
# whisper_df.to_csv('data/embeddings/whisper.csv', index=False)

MoviePy - Writing audio in temp_audio.wav



chunk:  66%|█████████████████████████████████████████████████████████████▋                               | 879/1324 [00:00<00:00, 8785.17it/s, now=None]
                                                                                                                                                        

MoviePy - Done.
MoviePy - Writing audio in temp_audio.wav



chunk:  95%|█████████████████████████████████████████████████████████████████████████████████████████▎    | 920/968 [00:00<00:00, 9193.41it/s, now=None]
                                                                                                                                                        

MoviePy - Done.
MoviePy - Writing audio in temp_audio.wav


...

MoviePy - Done.
MoviePy - Writing audio in temp_audio.wav



chunk:  69%|████████████████████████████████████████████████████████████████▍                            | 918/1324 [00:00<00:00, 9106.82it/s, now=None]
                                                                                                                                                        

MoviePy - Done.
MoviePy - Writing audio in temp_audio.wav



chunk:   0%|                                                                                                          | 0/859 [00:00<?, ?it/s, now=None]
                                                                                                                                                        

MoviePy - Done.


## Quality metrics

In [ ]:
def top_k_accuracy(target: np.ndarray, preds: np.ndarray, k: int = 10) -> float:
    numer = 0
    denom = target.size

    for label, labels_set in zip(target, preds):
        if label in labels_set[:k]:
            numer += 1

    return numer * 1.0 / denom

def string_to_numpy(tensor_string):
    tensor_string = tensor_string.replace('tensor(', '').rstrip(')')
    tensor_data = ast.literal_eval(tensor_string)
    tensor = torch.tensor(tensor_data)

    return tensor

def string_to_numpy(tensor_string):
    tensor_string = tensor_string.replace('tensor(', '').rstrip(')')
    tensor_data = ast.literal_eval(tensor_string)
    tensor_numpied = torch.tensor(tensor_data).numpy()

    return tensor_numpied


label_data = [
    (0, 'Авто-мото'),
    (1, 'Аниме'),
    (2, 'Аудиокниги'),
    (3, 'Бизнес'),
    (4, 'Видеоигры'),
    (5, 'Интервью'),
    (6, 'Искусство'),
    (7, 'Кино'),
    (8, 'Красота'),
    (9, 'Кулинария'),
    (10, 'Лайфхаки'),
    (11, 'Музыка'),
    (12, 'Мультфильмы'),
    (13, 'Новости'),
    (14, 'Обучение'),
    (15, 'Охота и рыбалка'),
    (16, 'Политика'),
    (17, 'Психология'),
    (18, 'Путешествия'),
    (19, 'Сериалы'),
    (20, 'Спорт'),
    (22, 'Юмор'),
    (25, 'Лайфстайл'),
    (26, 'Недвижимость'),
    (27, 'Здоровье'),
    (28, 'Природа'),
    (29, 'Дизайн'),
    (30, 'Техника и оборудование'),
    (31, 'Бизнес и предпринимательство'),
    (32, 'Культура'),
    (33, 'Религия'),
    (34, 'Строительство и ремонт'),
    (35, 'Сад и огород'),
    (36, 'Еда'),
    (37, 'Развлечения'),
    (38, 'Эзотерика'),
    (39, 'Наука'),
    (40, 'Аудио'),
    (41, 'Технологии и интернет'),
    (42, 'Телепередачи'),
    (43, 'Детям'),
    (44, 'Хобби'),
    (45, 'Разное'),
    (46, 'Животные'),
    (47, 'Новости и СМИ'),
    (48, 'Фильмы'),
    (49, 'Блогеры')
]
label_df = pd.DataFrame(label_data, columns=['tag_id', 'tag'])

## X-CLIP

### No context

In [ ]:
tag_id_to_index

In [ ]:
clip_df = pd.read_csv('data/embeddings/x-clip-v2.csv')
tag_id_to_index = dict(zip(label_df.tag_id, label_df.index))

Accuracy of first window

In [ ]:
clip_id_label = clip_df.label.apply(lambda x: tag_id_to_index[x]).values
top_n_predictions = np.vstack(clip_df.probs.apply(lambda x: np.argsort(string_to_numpy(x))[0, :][::-1]))

print('Top 1 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 1) * 100)
print('Top 5 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 5) * 100)
print('Top 10 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 10) * 100)

Top 1 accuracy: 8.16831683168317
Top 5 accuracy: 19.554455445544555
Top 10 accuracy: 32.17821782178218


Accuracy of second window

In [ ]:
clip_id_label = clip_df.label.apply(lambda x: tag_id_to_index[x]).values
top_n_predictions = np.vstack(clip_df.probs.apply(lambda x: np.argsort(string_to_numpy(x))[1, :][::-1]))

print('Top 1 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 1) * 100)
print('Top 5 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 5) * 100)
print('Top 10 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 10) * 100)

Top 1 accuracy: 9.158415841584159
Top 5 accuracy: 20.049504950495052
Top 10 accuracy: 31.18811881188119


Accuracy of third window

In [ ]:
clip_id_label = clip_df.label.apply(lambda x: tag_id_to_index[x]).values
top_n_predictions = np.vstack(clip_df.probs.apply(lambda x: np.argsort(string_to_numpy(x))[2, :][::-1]))

print('Top 1 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 1) * 100)
print('Top 5 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 5) * 100)
print('Top 10 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 10) * 100)

Top 1 accuracy: 5.445544554455446
Top 5 accuracy: 20.049504950495052
Top 10 accuracy: 30.198019801980198


Accuracy of average window

In [ ]:
clip_id_label = clip_df.label.apply(lambda x: tag_id_to_index[x]).values
top_n_predictions = np.vstack(clip_df.probs.apply(lambda x: np.argsort(string_to_numpy(x)).mean(axis=0)[::-1]))

print('Top 1 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 1) * 100)
print('Top 5 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 5) * 100)
print('Top 10 accuracy:', top_k_accuracy(clip_id_label, top_n_predictions, 10) * 100)

Top 1 accuracy: 3.9603960396039604
Top 5 accuracy: 6.9306930693069315
Top 10 accuracy: 10.891089108910892


### ~~Whisper huggingface context~~

We need to increase the maximum text size from 77 to 1024

So, we have to make some tricks

In [ ]:
POSITION_EMB_SIZE = 1024


model, processor, window_size = instantiate_model(model_name, device)
model.text_model.config.max_position_embeddings = POSITION_EMB_SIZE

old_embedding = model.text_model.embeddings.position_embedding
embedding_dim = old_embedding.weight.shape[1]
new_embedding = nn.Embedding(POSITION_EMB_SIZE, embedding_dim).to(device)

# Initialize the new embeddings with the old embeddings
num_tokens_to_copy = min(old_embedding.weight.shape[0], POSITION_EMB_SIZE)
new_embedding.weight.data[:num_tokens_to_copy, :] = old_embedding.weight.data[:num_tokens_to_copy, :]

# Replace the model's position embeddings with the new embeddings
model.text_model.embeddings.position_embedding = new_embedding

processor.tokenizer.model_max_length = POSITION_EMB_SIZE

In [ ]:
def extract_category_id(ctgry: str) -> int:
    return int(float(ctgry[:3]))


def extract_text_by_video_name(whisper_df: pd.DataFrame, videoname: str):
    if videoname not in whisper_df.videoname.values:
        print(f"Такого видео не существует: {videoname}")
        return None

    text = whisper_df[whisper_df.videoname == videoname]['text']

    if text is None or text is np.nan or text is pd.NA:
        return None
    else:
        return text.values[0]


def get_xclip_prompt_with_audio_context(whisper_df: pd.DataFrame, videoname: str):
    text = extract_text_by_video_name(whisper_df, videoname)

    if text is None:
        return RUS_TEXT_PROMPT
    else:
        return [f"Видео субтитры: '{text}'. " + x for x in RUS_TEXT_PROMPT]


def postprocess_model_and_processor(model, processor, device):
    POSITION_EMB_SIZE = 1024

    model, processor, window_size = instantiate_model(model_name, device)
    model.text_model.config.max_position_embeddings = POSITION_EMB_SIZE

    old_embedding = model.text_model.embeddings.position_embedding
    embedding_dim = old_embedding.weight.shape[1]
    new_embedding = nn.Embedding(POSITION_EMB_SIZE, embedding_dim).to(device)

    # Initialize the new embeddings with the old embeddings
    num_tokens_to_copy = min(old_embedding.weight.shape[0], POSITION_EMB_SIZE)
    new_embedding.weight.data[:num_tokens_to_copy, :] = old_embedding.weight.data[:num_tokens_to_copy, :]

    # Replace the model's position embeddings with the new embeddings
    model.text_model.embeddings.position_embedding = new_embedding
    processor.tokenizer.model_max_length = POSITION_EMB_SIZE

    return model, processor


DATA_DIR = './data'
model_name = 'x-clip'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model, processor, window_size = instantiate_model(model_name, device)
model, processor = postprocess_model_and_processor(model, processor, device)
embeddings_xclip = []

whisper_df = pd.read_csv('data/embeddings/whisper.csv')

for data_subdir in tqdm.tqdm(os.listdir(DATA_DIR)):
    fullpath = os.path.join(DATA_DIR, data_subdir)

    if not os.path.isdir(fullpath):
        continue

    video_files = [f for f in os.listdir(fullpath) if f.lower().endswith('.mp4')]
    logging.info(f"Found {len(video_files)} video(s) in directory '{data_subdir}'.")

    for video_file in video_files:
        file_path = os.path.join(fullpath, video_file)
        output = process_video(
            file_path,
            processor,
            model,
            device,
            desired_frames=96,
            window_size=window_size,
            xclip_text_prompt=get_xclip_prompt_with_audio_context(whisper_df, video_file)
        )

        if output is not None:
            logging.info(f"Successfully processed video: {video_file}")
            embeddings_xclip.append((video_file, extract_category_id(data_subdir), output[0], output[1]))
        else:
            logging.warning(f"Failed to process video: {video_file}")

xclip_df = pd.DataFrame(data=embeddings_xclip, columns=['videoname', 'label', 'emb', 'probs'])
xclip_df.to_csv('data/embeddings/x-clip-whisper.csv', index=False)

  0%|          | 0/48 [00:00<?, ?it/s]

True torch.Size([47, 398]) torch.Size([3, 32, 3, 224, 224])
An unexpected error occurred:
The size of tensor a (398) must match the size of tensor b (77) at non-singleton dimension 1


RuntimeError: The size of tensor a (398) must match the size of tensor b (77) at non-singleton dimension 1

In [ ]:
new_text_prompt = get_xclip_prompt_with_audio_context(whisper_df, "2d102489e381cb2dc28395d3a76771b1.mp4")
video_path = "./data/48.0_films/2d102489e381cb2dc28395d3a76771b1.mp4"

process_video(
    video_path,
    processor,
    model,
    device,
    xclip_text_prompt=new_text_prompt
)

True torch.Size([47, 398]) torch.Size([3, 32, 3, 224, 224])
An unexpected error occurred:
The size of tensor a (398) must match the size of tensor b (77) at non-singleton dimension 1


RuntimeError: The size of tensor a (398) must match the size of tensor b (77) at non-singleton dimension 1

### ~~Whisper API context~~

## CCA

In [ ]:
import numpy as np
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
whisper_df = pd.read_csv('data/embeddings/whisper.csv')
whisper_df.dropna(inplace=True)

# whisper_df['label_new'] = whisper_df.label.apply(lambda x: tag_id_to_index[x])
whisper_df_emb = np.vstack(whisper_df.emb.apply(lambda x: string_to_numpy(x)))
whisper_df_emb

IndentationError: unexpected indent (<unknown>, line 2)

In [ ]:
df = pd.DataFrame()
df['a'] = np.arange(10)
df['b'] = [np.random.rand(3, 1024) for _ in range(10)]
df.to_csv('temp.csv', index=False)

df_2 = pd.read_csv('temp.csv')
df_2['b'][0]

'[[0.37479353 0.42868598 0.68305662 ... 0.79010835 0.98197353 0.10749206]\n [0.84141261 0.48978955 0.05141087 ... 0.53338855 0.74019714 0.22805642]\n [0.30034419 0.19453843 0.44306205 ... 0.48031917 0.76141112 0.89792329]]'

In [ ]:
whisper_df = pd.read_csv('data/embeddings/whisper.csv')
whisper_df.dropna(inplace=True)
whisper_df['label_new'] = whisper_df.label.apply(lambda x: tag_id_to_index[x])
whisper_df_emb = np.random.rand(whisper_df.shape[0], 1024)
# np.vstack(whisper_df.emb.apply(lambda x: string_to_numpy(x))))

## 1-NN (with metric learning?)

## MLP

## Gradient boosting